# Canadian Credit Union Yelp and Asset Growth Project

Here we explore the correlation between Yelp! reviews and asset growth using various recurrent neural networks. From our experimentation with the data, and features, we found _ yielded the best accuracy of future asset growth. This prediction was generated using Yelp! reviews and sentiment analysis.


## Initial Set-Up

Here we will import the necessary libraries that we will need for the project. Additionally, will read in the data collected from various Canadian Credit Unions and their corresponding Yelp! reviews. 

In [ ]:
# Let's first start by importing the libraries and data we'll need
# Libraries needed include numpy, keras, matplotlib

import numpy as np
import keras
import pandas as pd
from matplotlib import pyplot as plt
import pickle
import SentimentAverages as sa

# Import data

data = pickle.load(open("dfFinal.p", "rb"))

print(data)

cudata = pd.read_excel("cuinfo.xlsx", sheet_name=None)

## Data Preparation 

Structure the data correctly as to be easily used by the generator and the keras model.

In [ ]:
print(list(cudata.keys()))
companies = list(data.columns.levels[0])
print(companies)

In [ ]:
indecies = ['Vancity','Coast Capital Savings','Servus Credit Union','Meridian Credit Union','Affinity Federal Credit Union']

print('Vancity')
vancity = sa.getSentimentAves('Vancity')
print(vancity)
print('Coast Capital Savings')
print(sa.getSentimentAves('Coast Capital Savings'))
print('Servus Credit Union')
print(sa.getSentimentAves('Servus Credit Union'))
print('Meridian Credit Union')
print(sa.getSentimentAves('Meridian Credit Union'))
print('Affinity Federal Credit Union')
print(sa.getSentimentAves('Affinity Federal Credit Union'))

In [ ]:
# Extract data from excel
finaldata = [0,1,2,3,4]
for company in companies:
    # Translate Names
    if company == "Vancity":
        header = "Vancity"
        index = 0
    elif company == "Coast Capital Savings":
        header = "Coast Capital"
        index = 1
    elif company == "Servus Credit Union":
        header = "Servus"
        index = 2
    elif company == "Meridian Credit Union":
        header = "Meridian"
        index = 3
    elif company == "Affinity Federal Credit Union":
        header = "Affinity"
        index = 4
    elif company == "Assiniboine Credit Union":
        continue
    # Extract from correct sheet
    one = pd.concat([cudata[header]["Year (Growth from X-1 to X)"], cudata[header]["Total Assets ($ CAD)"]], axis=1)
    #one.set_index(one.columns[0], inplace=True)
    one = one.dropna()
    one[one.columns[0]] = pd.to_numeric(one[one.columns[0]], downcast='integer')
    one.set_index(one.columns[0], inplace=True)
    
    toset = pd.concat([one,sa.getSentimentAves(company)], axis=1)
    
    finaldata[index] = toset.dropna().values

    print(header)
    print(finaldata[index])
    

## Data Analytics 

Here we will plot some of our data to see if we can see any obvious patterns. Best to do that before going right into the model creation so that we can ensure that the obvious patterns are indeed accounted for during that stage. Here will simply look at temperature versus time data.

In [ ]:
# Store our Vancity asset data into a numpy array for convenience

# vassets  = float_data[:,1] 
# time  = data[:,1]
# plt.plot(time, data, label = 'Asset Growth ($ CAD)')
# plt.title('Asset Growth By Year')
# plt.legend()
# plt.xlabel('Year')
# plt.ylabel('Asset Growth ($ CAD)')

As is evident from the plot above, we should be expecting year over year increase in assets. This will provide us with a diagnostic to ensure the model is providing us with reasonable output. Additionally, should get yearly growths in excess of a few hundreds of millions of dollars. This should pick up to a few billion dollars around the year 2000.

## Data Generation
Will now define a generator function to create our training, validation and testing data sets for our model. Given the nature of the data we are working with, will require more hardcoding than might typically be required.

In [ ]:
# Building the generator.  Hardcoded for our purpose

def train_gen(data):
    lookback = 5
    company = 0 #Counter for which company is being chosen
    i = 0 # Index in that company's dataframe
    
    while 1:
        while i + lookback < len(data[company]):
            samples = np.zeros((1,lookback,2))
            targets = np.zeros((1,))
            for k, j in enumerate(range(i, i + lookback)):
                # k in [0, lookback)
                # j in [i, i + lookback)
                for s in range(0,len(data[company][j])):
                    samples[0][k][s] = data[company][j][s]
            targets[0] = data[company][i + lookback][1]
            i = i + 1
            yield samples, targets
        i = 0
        if company < len(data) - 1:
            company += 1
        else:
            company = 0




def generator(data, lookback, delay , min_index, max_index, batch_size = 128):
    step = 1
    
    if max_index is None:
        max_index = len(data)  - delay - 1
    i = min_index + lookback
    
    while 1:
        if i + batch_size >= max_index:
            i = min_index + lookback
        rows = np.arange(i, min(i + batch_size, max_index))
        i += len(rows)
        samples = np.zeros((len(rows), lookback // step, data.shape[-1]))
        targets = np.zeros((len(rows), ))
        
        for j , row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]
        yield samples, targets

In [ ]:
# for i in range(0,5):
#     companytest = np.random.rand(25,3)
#     datatest.append(companytest)
# print(len(datatest), datatest[0].shape)

train_data = list(finaldata[i] for i in [0,1,3] )
#print(train_data)

val_data = [finaldata[2]]
#print(val_data)

test_data = [finaldata[4]]
print([test_data])


train = train_gen(train_data)
val = train_gen(val_data)
test = train_gen(test_data)

In [ ]:
print(np.shape(next(train)[0]), '\n')
print(next(val), '\n')
print(next(test))

## Model Creation

Finally, after all the hard work of normalizing our data, taking a quick look at it, we get the to fun part: model creation. Will be taking Yelp! reviews and provided asset growth data to predict future asset growth as our only output. This will be accomplished using an LSTM recurrent neural network model with _ layers and _ neurons. These were selected as they yielded the highest accuracy from our experimentation. 

In [ ]:
# Need to import some libraries from keras to create our model
# This will involve the use of keras sequential neural network models, layers and rmsprop optimizers

from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop
from keras.layers.recurrent import LSTM

# The rest is very similar to the creation of the sequential neural network we made during the Warm-Up project
# Quick refresher though, need to define our model with a number of layers, an optimzer function, a loss function, an activation function, and how many layers we want it to be

# Define our model as a sequential one
model = Sequential()

# Add some layers to our model 
#model.add(LSTM(5, input_shape=(5, 2), return_sequences = True))
model.add(layers.Flatten(input_shape=(5,2)))
model.add(layers.Dense(32, activation = 'relu'))
model.add(layers.Dense(1))

# Now compile our model with optimizer and loss functions, no metric for this one though
model.compile(optimizer = RMSprop(), loss = 'mae')
history =  model.fit_generator(train,
                               epochs = 3,
                               steps_per_epoch = 5,
                               validation_data = val,
                               validation_steps = 1)

# Before we go any further, some important notes to make here. Will do that below in the "Model Notes" block

## Model Notes

1. Could have used other activation functions, 'relu' is a pretty popular one, but could use the likes of 'selu' and 'sigmoid'.

2. The number of layers we add is completely arbitrary and is usually driven by experimenting with the model to see what works the best for the project.

3. The number of epochs is another great place to play around. This is primarily due to wanting to avoid overfitting, which can happen by having too many training epochs. As such, should play around and see how many epochs yields the best result for the model.

4. The optimizer function is another area to play around as RMSprop may not always be the best choice for the project at hand.

5. The loss function selected here was another judgement call, but others could be used such as binary cross entropy. Used here since we actually have numbers to match to our model's prediction, so makes sense to use mean absolute error to see how far away our model's predictions are so we can mitigate the errors. Could also use root mean square method as well for the same purpose.

## Model Evaluation

Now that we have trained and validated our model, it is imperative we now test it. This will be done using model.evaluate() and the test data we set aside for it earlier.

In [ ]:
results = model.evaluate_generator(test, steps = 3)

#Print out the accuracy of our model on the testing data

print("Accuracy:", results)

## Model Output

Will now plot our model's prediction against the actual data, and validation data to see if we're overfitting, and how our model is performing overall.

In [ ]:
# Will grab our losses by going into the training history and defining appropriate variables to make plotting easier

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

# Now plot our training and validation losses

plt.figure()
plt.plot(epochs, loss, 'b', label = 'Training Loss')
plt.plot(epochs, val_loss, 'r', label = 'Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Model Training and Validation Losses By Epoch')
plt.legend()
plt.show()

In [ ]:
# Will now have our model make a prediction
# X will be our input that we give to our model to make its prediction

prediction = model.predict_generator(test, steps = 1)
print(prediction)

## Model Evaluation

As is evident from the plot and historical output above, our model is able to achieve accuracy in the range of _%-_%. Furthermore, it is evident that in using _ epochs, the model is able to avoid overfitting to the training data. As such, it can be concluded that our model is sufficiently effective in predicting future asset growth for Canadian Credit Unions using Yelp! reviews, and yearly asset growth. Lastly, it can be concluded that Vancity Credit Uni